In [ ]:
from ib_insync import *
import pandas as pd
from datetime import datetime, timedelta, time

In [ ]:
D2 = datetime(2023,5,8)
market_close = time(16,0,1)
end_datetime = datetime.combine(D2, market_close)
print(end_datetime)

In [ ]:


from ib_insync import *
import pandas as pd

# Connect to TWS or IB Gateway (ensure it's running)
ib = IB()
await ib.connectAsync('127.0.0.1', 7497, clientId=13)  # use a unique clientId if needed

# Define the contract (you can change the symbol here)
contract = Stock('GOOG', 'SMART', 'USD')

# Request 30 minutes of 1-second bars ending now
bars = await ib.reqHistoricalDataAsync(
    contract,
    endDateTime=end_datetime.strftime('%Y%m%d %H:%M:%S'),                 # '' = now
    durationStr='602 S',          # 1800 seconds = 30 minutes
    barSizeSetting='1 secs',       # <-- 1-second bars
    whatToShow='TRADES',
    useRTH=False,                  # False = include pre/post market
    formatDate=1
)

# Disconnect from IBKR
ib.disconnect()

# Convert to DataFrame
new_df = util.df(bars)
print(f"✅ Retrieved {len(new_df)} 1-second bars")
display(new_df.head())


loop over many days

In [ ]:
from ib_insync import *
import pandas as pd
from datetime import datetime, timedelta, time
import pytz
import asyncio
import pandas_market_calendars as mcal

# Step 1: Connect to IBKR
ib = IB()
await ib.connectAsync('127.0.0.1', 7497, clientId=19)

# Step 2: Define timezone and trading calendar
#eastern = pytz.timezone('US/Eastern')
market_close_time = time(16,0,1)
start_cutoff = datetime(2022, 7, 1).date()
end_cutoff = datetime(2025, 7, 7).date()
#today = datetime.now(tz=eastern).date()

# Step 3: Get valid NYSE trading days
nyse = mcal.get_calendar('NYSE')
schedule = nyse.schedule(start_date=start_cutoff, end_date=end_cutoff)
valid_dates = list(schedule.index.date)[::-1]  # Reverse for backward loop

# Step 4: Request loop
all_data = []

contract = Stock('GOOG', 'SMART', 'USD')

for trading_day in valid_dates:
    # Compose datetime ending at 4:00 PM
    print(f"the trading_day is: {trading_day}")
    end_dt = datetime.combine(trading_day, market_close_time)
    print(f"the end_dt is: {end_dt}")
    # end_dt_eastern = eastern.localize(end_dt)
    # print(f"the end_dt_eastern is: {end_dt_eastern}")
    # print(f"the UTC is: {end_dt_eastern.astimezone(pytz.utc)}")

    try:
        print(f"⏳ Requesting {trading_day}...")
        bars = await ib.reqHistoricalDataAsync(
            contract,
            endDateTime=end_dt.strftime('%Y%m%d %H:%M:%S'),
            durationStr='602 S',               # 10 minutes = 3:50PM to 4:00PM
            barSizeSetting='1 secs',
            whatToShow='TRADES',
            useRTH=True,                       # Only regular trading hours
            formatDate=1
        )

        if bars:
            df = util.df(bars)
            #display(df.head())
            if not df.empty and df['date'].iloc[1].time() >= time(15,49,59):
                print(f"✅ {trading_day}: {len(df)} rows")
                df['date'] = pd.to_datetime(df['date']).dt.tz_localize(None)
                df['day'] = trading_day
                all_data.append(df)
            else:
                print(f"⚠️ {trading_day}: No data or market closed early")
        else:
            print(f"bars is empty for {trading_day}")
    except Exception as e:
        print(f"❌ Error on {trading_day}: {e}")

    await asyncio.sleep(11)  # Respect IBKR rate limits

# Step 5: Disconnect
ib.disconnect()

# Step 6: Combine and Save
final_df = pd.concat(all_data)
print(f"\n📊 Final dataset contains {len(final_df)} rows across {final_df['day'].nunique()} trading days")
final_df.reset_index(drop=True, inplace=True)
final_df.to_csv("GOOG_1s_3_50PM_to_4_00PM.csv", index=False)
